# 第8章 文埋め込み

## 8.3 文埋め込みモデルの実装

### 8.3.1 教師なしSimCSEの実装

#### 準備

In [1]:
# 必要なパッケージをインストールする
!pip install datasets scipy transformers[ja,torch]

In [2]:
from transformers.trainer_utils import set_seed

# 乱数のシードを設定する
set_seed(42)

/home/studio-lab-user/.conda/envs/llm-book/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### データセットの読み込みと前処理

In [3]:
from datasets import load_dataset

# Hugging Face Hubのllm-book/jawiki-sentencesのリポジトリから
# Wikipediaの文のデータを読み込み、SimCSEの訓練セットとして使用する
unsup_train_dataset = load_dataset(
    "llm-book/jawiki-sentences", split="train"
)

/home/studio-lab-user/.conda/envs/llm-book/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for llm-book/jawiki-sentences contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/llm-book/jawiki-sentences
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating train split: 100%|██████████| 24387500/24387500 [00:55<00:00, 437333.14 examples/s]


In [4]:
# 訓練セットの形式と事例数を確認する
print(unsup_train_dataset)

Dataset({
    features: ['text'],
    num_rows: 24387500
})


In [5]:
# 訓練セットの内容を確認する
for i, text in enumerate(unsup_train_dataset[:50]["text"]):
    print(i, text)

0 アンパサンド(&, 英語: ampersand)は、並立助詞「...と...」を意味する記号である。
1 ラテン語で「...と...」を表す接続詞 "et" の合字を起源とする。
2 現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" の合字であることが容易にわかる字形を使用している。
3 英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字("A", "I", かつては "O" も)については、伝統的にラテン語の per se(それ自体)を用いて "A per se A" のように唱えられていた。
4 また、アルファベットの最後に、27番目の文字のように "&" を加えることも広く行われていた。
5 "&" はラテン語で et と読まれていたが、後に英語で and と読まれるようになった。
6 結果として、アルファベットの復唱の最後は "X, Y, Z, and per se and" という形になった。
7 この最後のフレーズが繰り返されるうちに "ampersand" と訛っていき、この言葉は1837年までには英語の一般的な語法となった。
8 アンドレ=マリ・アンペールがこの記号を自身の著作で使い、これが広く読まれたため、この記号が "Ampère's and" と呼ばれるようになったという誤った語源俗説がある。
9 アンパサンドの起源は1世紀の古ローマ筆記体にまで遡ることができる。
10 古ローマ筆記体では、E と T はしばしば合字として繋げて書かれていた(左図「アンパサンドの変遷」の字形1)。それに続く、流麗さを増した新ローマ筆記体では、様々な合字が極めて頻繁に使われるようになった。
11 字形2と3は4世紀中頃における et の合字の例である。
12 その後、9世紀のカロリング小文字体に至るラテン文字の変遷の過程で、合字の使用は一般には廃れていった。
13 しかし、et の合字は使われ続け、次第に元の文字がわかりにくい字形に変化していった(字形4から6)。
14 現代のイタリック体のアンパサンドは、ルネサンス期に発展した筆記体での et の合字に遡る。
15 1455年のヨーロッパにおける印刷技術の発明以降、印刷業者はイタリック体とローマ筆記体のアンパサンドの両方を多用するようになった。
16

In [6]:
# 訓練セットから空行の事例を除外する
unsup_train_dataset = unsup_train_dataset.filter(
    lambda example: example["text"].strip() != ""
)

Filter: 100%|██████████| 24387500/24387500 [01:01<00:00, 397051.11 examples/s]


In [7]:
# 訓練セットをシャッフルし、最初の100万事例を取り出す
unsup_train_dataset = unsup_train_dataset.shuffle().select(
    range(1000000)
)
# パフォーマンスの低下を防ぐため、シャッフルされた状態の訓練セットを
# ディスクに書き込む
unsup_train_dataset = unsup_train_dataset.flatten_indices()

Flattening the indices: 100%|██████████| 1000000/1000000 [00:10<00:00, 98882.24 examples/s]


In [8]:
# 前処理後の訓練セットの形式と事例数を確認する
print(unsup_train_dataset)

Dataset({
    features: ['text'],
    num_rows: 1000000
})


In [9]:
# 前処理後の訓練セットの内容を確認する
for i, text in enumerate(unsup_train_dataset[:10]["text"]):
    print(i, text)

0 2005年の時点で、10,000人ものウズベキスタン人が韓国での労働に従事しており、その大部分が高麗人である。
1 小学5年生(11歳)の時から芸能活動を開始。
2 i ħ d d t | ψ ( t ) ⟩ = L ^ | ψ ( t ) ⟩ {\displaystyle i\hbar {\frac {d}{dt}}|\psi (t)\rangle ={\hat {L}}|\psi (t)\rangle }
3 安土宗論(あづちしゅうろん)は、1579年(天正7年)、安土城下の浄厳院で行われた浄土宗と法華宗の宗論。
4 1927年 オーストラリア選手権(1927ねんオーストラリアせんしゅけん、1927 Australian Championships)に関する記事。
5 さらにマップ上で最大8つまでしか建築できず(司令官アビリティの”解体”か設置したプレイヤー自らが出向いて解体する必要がある)
6 特に誉淳が1827年から作成した『古瓦譜』は畿内で600点以上の拓本を蒐集し、瓦当文様に着目したうえで編年を試みている。
7 マルクス主義者を広言し、メキシコ共産党の敵であり味方であった。
8 ICHILLIN'(アイチリン、朝: 아이칠린)は、韓国の7人組女性アイドルグループ。
9 マークVIは1983年にモデルサイクルを終了し、1984年のマークVII(英語版)はフルサイズセグメントから撤退し、マークシリーズは異なるセグメントに移行した。


In [10]:
# Hugging Face Hubのllm-book/JGLUEのリポジトリから
# JSTSデータセットの訓練セットと検証セットを読み込み、
# それぞれをSimCSEの検証セットとテストセットとして使用する
valid_dataset = load_dataset(
    "llm-book/JGLUE", name="JSTS", split="train"
)
test_dataset = load_dataset(
    "llm-book/JGLUE", name="JSTS", split="validation"
)

/home/studio-lab-user/.conda/envs/llm-book/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for llm-book/JGLUE contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/llm-book/JGLUE
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating train split: 12451 examples [00:00, 28579.70 examples/s]
Generating validation split: 1457 examples [00:00, 27840.75 examples/s]


#### トークナイザと collate 関数の準備

In [11]:
from transformers import AutoTokenizer

# Hugging Face Hubにおけるモデル名を指定する
base_model_name = "cl-tohoku/bert-base-japanese-v3"
# モデル名からトークナイザを初期化する
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

In [12]:
import torch
from torch import Tensor
from transformers import BatchEncoding

def unsup_train_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """教師なしSimCSEの訓練セットのミニバッチを作成"""
    # ミニバッチに含まれる文にトークナイザを適用する
    tokenized_texts = tokenizer(
        [example["text"] for example in examples],
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )

    # 文と文の類似度行列における正例ペアの位置を示すTensorを作成する
    # 行列のi行目の事例（文）に対してi列目の事例（文）との組が正例ペアとなる
    labels = torch.arange(len(examples))

    return {
        "tokenized_texts_1": tokenized_texts,
        "tokenized_texts_2": tokenized_texts,
        "labels": labels,
    }

In [13]:
def eval_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """SimCSEの検証・テストセットのミニバッチを作成"""
    # ミニバッチの文ペアに含まれる文（文1と文2）のそれぞれに
    # トークナイザを適用する
    tokenized_texts_1 = tokenizer(
        [example["sentence1"] for example in examples],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    tokenized_texts_2 = tokenizer(
        [example["sentence2"] for example in examples],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )

    # 文1と文2の類似度行列における正例ペアの位置を示すTensorを作成する
    # 行列のi行目の事例（文1）に対して
    # i列目の事例（文2）との組が正例ペアとなる
    labels = torch.arange(len(examples))

    # データセットに付与された類似度スコアのTensorを作成する
    label_scores = torch.tensor(
        [example["label"] for example in examples]
    )

    return {
        "tokenized_texts_1": tokenized_texts_1,
        "tokenized_texts_2": tokenized_texts_2,
        "labels": labels,
        "label_scores": label_scores,
    }

#### モデルの準備

In [14]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel
from transformers.utils import ModelOutput

class SimCSEModel(nn.Module):
    """SimCSEのモデル"""

    def __init__(
        self,
        base_model_name: str,
        mlp_only_train: bool = False,
        temperature: float = 0.05,
    ):
        """モデルの初期化"""
        super().__init__()

        # モデル名からエンコーダを初期化する
        self.encoder = AutoModel.from_pretrained(base_model_name)

        # MLP層の次元数
        self.hidden_size = self.encoder.config.hidden_size
        # MLP層の線形層
        self.dense = nn.Linear(self.hidden_size, self.hidden_size)
        # MLP層の活性化関数
        self.activation = nn.Tanh()

        # MLP層による変換を訓練時にのみ適用するよう設定するフラグ
        self.mlp_only_train = mlp_only_train
        # 交差エントロピー損失の計算時に使用する温度
        self.temperature = temperature

    def encode_texts(self, tokenized_texts: BatchEncoding) -> Tensor:
        """エンコーダを用いて文をベクトルに変換"""
        # トークナイズされた文をエンコーダに入力する
        encoded_texts = self.encoder(**tokenized_texts)
        # モデルの最終層の出力（last_hidden_state）の
        # [CLS]トークン（0番目の位置のトークン）のベクトルを取り出す
        encoded_texts = encoded_texts.last_hidden_state[:, 0]

        # self.mlp_only_trainのフラグがTrueに設定されていて
        # かつ訓練時でない場合、MLP層の変換を適用せずにベクトルを返す
        if self.mlp_only_train and not self.training:
            return encoded_texts

        # MLP層によるベクトルの変換を行う
        encoded_texts = self.dense(encoded_texts)
        encoded_texts = self.activation(encoded_texts)

        return encoded_texts

    def forward(
        self,
        tokenized_texts_1: BatchEncoding,
        tokenized_texts_2: BatchEncoding,
        labels: Tensor,
        label_scores: Tensor | None = None,
    ) -> ModelOutput:
        """モデルの前向き計算を定義"""
        # 文ペアをベクトルに変換する
        encoded_texts_1 = self.encode_texts(tokenized_texts_1)
        encoded_texts_2 = self.encode_texts(tokenized_texts_2)

        # 文ペアの類似度行列を作成する
        sim_matrix = F.cosine_similarity(
            encoded_texts_1.unsqueeze(1),
            encoded_texts_2.unsqueeze(0),
            dim=2,
        )

        # 交差エントロピー損失を求める
        loss = F.cross_entropy(sim_matrix / self.temperature, labels)

        # 性能評価に使用するため、正例ペアに対するスコアを類似度行列から取り出す
        positive_mask = F.one_hot(labels, sim_matrix.size(1)).bool()
        positive_scores = torch.masked_select(
            sim_matrix, positive_mask
        )

        return ModelOutput(loss=loss, scores=positive_scores)

# 教師なしSimCSEのモデルを初期化する
unsup_model = SimCSEModel(base_model_name, mlp_only_train=True)

#### `Trainer` の準備

In [15]:
from scipy.stats import spearmanr
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction) -> dict[str, float]:
    """
    モデルが予測したスコアと評価用データのスコアの
    スピアマンの順位相関係数を計算
    """
    scores = p.predictions
    labels, label_scores = p.label_ids

    spearman = spearmanr(scores, label_scores).statistic

    return {"spearman": spearman}

In [16]:
from transformers import TrainingArguments

# 教師なしSimCSEの訓練のハイパーパラメータを設定する
unsup_training_args = TrainingArguments(
    output_dir="outputs_unsup_simcse",  # 結果の保存先フォルダ
    per_device_train_batch_size=64,  # 訓練時のバッチサイズ
    per_device_eval_batch_size=64,  # 評価時のバッチサイズ
    learning_rate=3e-5,  # 学習率
    num_train_epochs=1,  # 訓練エポック数
    evaluation_strategy="steps",  # 検証セットによる評価のタイミング
    eval_steps=250,  # 検証セットによる評価を行う訓練ステップ数の間隔
    logging_steps=250,  # ロギングを行う訓練ステップ数の間隔
    save_steps=250,  # チェックポイントを保存する訓練ステップ数の間隔
    save_total_limit=1,  # 保存するチェックポイントの最大数
    fp16=True,  # 自動混合精度演算の有効化
    load_best_model_at_end=True,  # 最良のモデルを訓練終了後に読み込むか
    metric_for_best_model="spearman",  # 最良のモデルを決定する評価指標
    remove_unused_columns=False,  # データセットの不要フィールドを削除するか
)

In [17]:
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import Trainer

class SimCSETrainer(Trainer):
    """SimCSEの訓練に使用するTrainer"""

    def get_eval_dataloader(
        self, eval_dataset: Dataset | None = None
    ) -> DataLoader:
        """
        検証・テストセットのDataLoaderでeval_collate_fnを使うように
        Trainerのget_eval_dataloaderをオーバーライド
        """
        if eval_dataset is None:
            eval_dataset = self.eval_dataset

        return DataLoader(
            eval_dataset,
            batch_size=64,
            collate_fn=eval_collate_fn,
            pin_memory=True,
        )

# 教師なしSimCSEのTrainerを初期化する
unsup_trainer = SimCSETrainer(
    model=unsup_model,
    args=unsup_training_args,
    data_collator=unsup_train_collate_fn,
    train_dataset=unsup_train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

/home/studio-lab-user/.conda/envs/llm-book/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 4.14.336, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


#### 訓練の実行

In [18]:
# 教師なしSimCSEの訓練を行う
unsup_trainer.train()

Step,Training Loss,Validation Loss,Spearman
250,0.001600,2.630450,0.700186
500,0.000200,2.524274,0.721852
750,0.000200,2.468607,0.732345
1000,0.000200,2.370710,0.751188
1250,0.000800,2.403104,0.753607
1500,0.000100,2.427210,0.754809
1750,0.000200,2.510145,0.750188
2000,0.000100,2.472742,0.756148
2250,0.000100,2.390910,0.764545
2500,0.000400,2.427595,0.755972


TrainOutput(global_step=15625, training_loss=0.00014509304231405257, metrics={'train_runtime': 4436.2037, 'train_samples_per_second': 225.418, 'train_steps_per_second': 3.522, 'total_flos': 0.0, 'train_loss': 0.00014509304231405257, 'epoch': 1.0})

#### 性能評価

In [19]:
# 検証セットで教師なしSimCSEのモデルの評価を行う
unsup_trainer.evaluate(valid_dataset)

{'eval_loss': 2.3518168926239014,
 'eval_spearman': 0.7656043464461134,
 'eval_runtime': 13.8005,
 'eval_samples_per_second': 902.215,
 'eval_steps_per_second': 14.13,
 'epoch': 1.0}

In [20]:
# テストセットで教師なしSimCSEのモデルの評価を行う
unsup_trainer.evaluate(test_dataset)

{'eval_loss': 2.2372379302978516,
 'eval_spearman': 0.7926071924855492,
 'eval_runtime': 1.8144,
 'eval_samples_per_second': 803.028,
 'eval_steps_per_second': 12.676,
 'epoch': 1.0}

#### トークナイザとモデルの保存

In [21]:
# 教師なしSimCSEのエンコーダを保存
encoder_path = "outputs_unsup_simcse/encoder"
unsup_model.encoder.save_pretrained(encoder_path)
tokenizer.save_pretrained(encoder_path)

('outputs_unsup_simcse/encoder/tokenizer_config.json',
 'outputs_unsup_simcse/encoder/special_tokens_map.json',
 'outputs_unsup_simcse/encoder/vocab.txt',
 'outputs_unsup_simcse/encoder/added_tokens.json')

#### Google ドライブへの保存

In [ ]:
from google.colab import drive

# Googleドライブをマウントする
drive.mount("drive")

Mounted at drive


In [ ]:
# 保存されたモデルをGoogleドライブのフォルダにコピーする
!mkdir -p drive/MyDrive/llm-book
!cp -r outputs_unsup_simcse drive/MyDrive/llm-book

### 8.3.2 教師あり SimCSE の実装

#### 準備

In [22]:
# 乱数のシードを設定する
set_seed(42)

#### データセットの読み込みと前処理

In [23]:
# Hugging Face Hubのllm-book/jsnliのリポジトリから
# JSNLIの訓練セットを読み込む
jsnli_dataset = load_dataset("llm-book/jsnli", split="train")

/home/studio-lab-user/.conda/envs/llm-book/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for llm-book/jsnli contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/llm-book/jsnli
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating validation split: 100%|██████████| 3916/3916 [00:00<00:00, 40051.15 examples/s]


In [24]:
# JSNLIの訓練セットの形式と事例数を確認する
print(jsnli_dataset)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 533005
})


In [25]:
from pprint import pprint

# JSNLIの訓練セットの内容を確認する
pprint(jsnli_dataset[0])
pprint(jsnli_dataset[1])

{'hypothesis': '男 は 魔法 の ショー の ため に ナイフ を 投げる 行為 を 練習 して い ます 。',
 'label': 'neutral',
 'premise': 'ガレージ で 、 壁 に ナイフ を 投げる 男 。'}
{'hypothesis': '女性 が 畑 で 踊って い ます 。',
 'label': 'contradiction',
 'premise': '茶色 の ドレス を 着た 女性 が ベンチ に 座って い ます 。'}


In [26]:
import csv
import random
from typing import Iterator

# JSNLIの訓練セットから、前提文とラベルごとに仮説文をまとめたdictを作成する
premise2hypotheses = {}

premises = jsnli_dataset["premise"]  # 前提文
hypotheses = jsnli_dataset["hypothesis"]  # 仮説文
labels = jsnli_dataset["label"]  # ラベル

for premise, hypothesis, label in zip(premises, hypotheses, labels):
    if premise not in premise2hypotheses:
        premise2hypotheses[premise] = {
            "entailment": [],
            "neutral": [],
            "contradiction": [],
        }

    premise2hypotheses[premise][label].append(hypothesis)

In [27]:
def generate_sup_train_example() -> Iterator[dict[str, str]]:
    """教師ありSimCSEの訓練セットの事例を生成"""
    # JSNLIのデータから (前提文,「含意」ラベルの仮説文,「矛盾」ラベルの仮説文)
    # の三つ組を生成する
    for premise, hypotheses in premise2hypotheses.items():
        # 「矛盾」ラベルの仮説文が一つもない事例はスキップする
        if len(hypotheses["contradiction"]) == 0:
            continue

        # 「含意」ラベルの仮説文一つにつき、「矛盾」ラベルの仮説文一つを
        # ランダムに関連付ける
        for entailment_hypothesis in hypotheses["entailment"]:
            contradiction_hypothesis = random.choice(
                hypotheses["contradiction"]
            )
            # (前提文,「含意」ラベルの仮説文,「矛盾」ラベルの仮説文) の三つ組を
            # dictとして生成する
            yield {
                "premise": premise,
                "entailment_hypothesis": entailment_hypothesis,
                "contradiction_hypothesis": contradiction_hypothesis,
            }

# 定義したジェネレータ関数を用いて、教師ありSimCSEの訓練セットを構築する
sup_train_dataset = Dataset.from_generator(generate_sup_train_example)

Generating train split: 173438 examples [00:01, 135131.51 examples/s]


In [28]:
# 訓練セットの形式と事例数を確認する
print(sup_train_dataset)

Dataset({
    features: ['premise', 'entailment_hypothesis', 'contradiction_hypothesis'],
    num_rows: 173438
})


In [29]:
# 訓練セットの内容を確認する
pprint(sup_train_dataset[0])
pprint(sup_train_dataset[1])

{'contradiction_hypothesis': '男 が 台所 の テーブル で 本 を 読んで い ます 。',
 'entailment_hypothesis': 'ガレージ に 男 が い ます 。',
 'premise': 'ガレージ で 、 壁 に ナイフ を 投げる 男 。'}
{'contradiction_hypothesis': '黒人 は デスクトップ コンピューター を 使用 し ます 。',
 'entailment_hypothesis': '人 は 椅子 に 座って い ます 。',
 'premise': 'ラップ トップ コンピューター を 使用 して 机 に 座って いる 若い 白人 男 。'}


#### collate 関数の準備

In [30]:
def sup_train_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """訓練セットのミニバッチを作成"""
    premises = []
    hypotheses = []

    for example in examples:
        premises.append(example["premise"])

        entailment_hypothesis = example["entailment_hypothesis"]
        contradiction_hypothesis = example["contradiction_hypothesis"]

        hypotheses.extend(
            [entailment_hypothesis, contradiction_hypothesis]
        )

    # ミニバッチに含まれる前提文と仮説文にトークナイザを適用する
    tokenized_premises = tokenizer(
        premises,
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )
    tokenized_hypotheses = tokenizer(
        hypotheses,
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )

    # 前提文と仮説文の類似度行列における正例ペアの位置を示すTensorを作成する
    # 行列のi行目の事例（前提文）に対して
    # 2*i列目の要素（仮説文）が正例ペアとなる
    labels = torch.arange(0, 2 * len(premises), 2)

    return {
        "tokenized_texts_1": tokenized_premises,
        "tokenized_texts_2": tokenized_hypotheses,
        "labels": labels,
    }

#### モデルの準備

In [31]:
# 教師ありSimCSEのモデルを初期化する
sup_model = SimCSEModel(base_model_name, mlp_only_train=False)

#### `Trainer` の準備

In [32]:
# 教師ありSimCSEの訓練のハイパーパラメータを設定する
sup_training_args = TrainingArguments(
    output_dir="outputs_sup_simcse",  # 結果の保存先フォルダ
    per_device_train_batch_size=128,  # 訓練時のバッチサイズ
    per_device_eval_batch_size=128,  # 評価時のバッチサイズ
    learning_rate=5e-5,  # 学習率
    num_train_epochs=3,  # 訓練エポック数
    evaluation_strategy="steps",  # 検証セットによる評価のタイミング
    eval_steps=250,  # 検証セットによる評価を行う訓練ステップ数の間隔
    logging_steps=250,  # ロギングを行う訓練ステップ数の間隔
    save_steps=250,  # チェックポイントを保存する訓練ステップ数の間隔
    save_total_limit=1,  # 保存するチェックポイントの最大数
    fp16=True,  # 自動混合精度演算の有効化
    load_best_model_at_end=True,  # 最良のモデルを訓練終了後に読み込むか
    metric_for_best_model="spearman",  # 最良のモデルを決定する評価指標
    remove_unused_columns=False,  # データセットの不要フィールドを削除するか
)

In [33]:
# 教師ありSimCSEのTrainerを初期化する
sup_trainer = SimCSETrainer(
    model=sup_model,
    args=sup_training_args,
    data_collator=sup_train_collate_fn,
    train_dataset=sup_train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

/home/studio-lab-user/.conda/envs/llm-book/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 4.14.336, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


#### 訓練の実行

In [34]:
# 教師ありSimCSEの訓練を行う
sup_trainer.train()

Step,Training Loss,Validation Loss,Spearman
250,1.445300,2.753861,0.791452
500,1.093800,2.757966,0.783126
750,1.023000,2.760353,0.783092
1000,0.965800,2.830193,0.795144
1250,0.923500,2.794023,0.798560
1500,0.829300,2.847800,0.793646
1750,0.764000,2.883723,0.800975
2000,0.747500,2.895309,0.800955
2250,0.744400,2.898232,0.794120
2500,0.729000,2.895850,0.799306


TrainOutput(global_step=4065, training_loss=0.8101340421744672, metrics={'train_runtime': 2355.0691, 'train_samples_per_second': 220.934, 'train_steps_per_second': 1.726, 'total_flos': 0.0, 'train_loss': 0.8101340421744672, 'epoch': 3.0})

#### 性能評価

In [35]:
# 検証セットで教師ありSimCSEのモデルの評価を行う
sup_trainer.evaluate(valid_dataset)

{'eval_loss': 2.8837225437164307,
 'eval_spearman': 0.80097545647758,
 'eval_runtime': 13.9724,
 'eval_samples_per_second': 891.116,
 'eval_steps_per_second': 7.014,
 'epoch': 3.0}

In [36]:
# テストセットで教師ありSimCSEのモデルの評価を行う
sup_trainer.evaluate(test_dataset)

{'eval_loss': 2.607274293899536,
 'eval_spearman': 0.8199627743842093,
 'eval_runtime': 1.7989,
 'eval_samples_per_second': 809.959,
 'eval_steps_per_second': 6.671,
 'epoch': 3.0}

#### Google ドライブへの保存

In [ ]:
# 保存されたモデルをGoogleドライブのフォルダにコピーする
!cp -r outputs_sup_simcse drive/MyDrive/llm-book